<a href="https://colab.research.google.com/github/Krink22/AbandonedPropertySearch/blob/main/Parity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#Install ArcGIS API once (comment out after)
!pip install arcgis #install ArcGIS API for Python and its dependencies in the current Google Colab environment. ! is necessary to run shell commands in a Colab notebook.
print("Done")

Done


In [39]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [40]:
gis = GIS() #Anonymous connection to GIS

In [70]:
#VACANCY LAYER
#Note: You can get the desired layer url by clicking the ... next to the layer on codemap layer list and clicking description. The resulting arcgis url is what you need
maxRecs = 100000

#Connect to feature layer of interest and view fields
url = 'https://geodata.baltimorecity.gov/egis/rest/services/Housing/dmxLandPlanning/MapServer/34'
vacancy_feature_layer = FeatureLayer(url)
props = vacancy_feature_layer.properties
print(f'{props.name} (CodeMap Layer: {props.id})')

# Query the layer to get all features (records)
query_result = vacancy_feature_layer.query(where="1=1", out_fields="*", result_record_count = maxRecs, return_all_records = False)
vacancyDF = query_result.df #convert to df
vacancyCount = vacancyDF["NT"].value_counts()["Vacant"]

if vacancyDF.shape[0] >= maxRecs:
  print('WARNING: MAY BE MISSING SOME ENTRIES BECAUSE QUERY LIMIT REACHED (increase maxRecs to avoid this.')
print('Note: BLOCKLOT uniquely identifies a lot by its block number and lot number on that block')
print(f'{vacancyDF.shape=}')
print(f'{vacancyCount=}')
print(f'{vacancyDF.columns}')
print('unique blocklots: ' + str(len(vacancyDF['BLOCKLOT'].unique())))
print('total entries: ' + str(len(vacancyDF['BLOCKLOT'])))
vacancyDF.head()

All Vacant Building Notices (CodeMap Layer: 34)
Note: BLOCKLOT uniquely identifies a lot by its block number and lot number on that block
vacancyDF.shape=(13750, 13)
vacancyCount=13750
Index(['NoticeNum', 'BLOCKLOT', 'DateNotice', 'DateCancel', 'DateAbate', 'NT',
       'Council_District', 'OBJECTID', 'Address', 'Neighborhood', 'Owner_ABBR',
       'HousingMarketTypology2017', 'SHAPE'],
      dtype='object')
unique blocklots: 13750
total entries: 13750


,NoticeNum,BLOCKLOT,DateNotice,DateCancel,DateAbate,NT,Council_District,OBJECTID,Address,Neighborhood,Owner_ABBR,HousingMarketTypology2017,SHAPE
0,805231A,0001 003,2012-01-25 15:37:00,NaT,NaT,Vacant,7,723192,2041 W NORTH AVE,EASTERWOOD,<NA>,I,"{""x"": 1411074.198864475, ""y"": 598468.355480149..."
1,1780434A,0001 004,2019-04-20 08:55:00,NaT,NaT,Vacant,7,723193,2039 W NORTH AVE,EASTERWOOD,<NA>,I,"{""x"": 1411088.4324318022, ""y"": 598469.08743405..."
2,927919A,0001 007,2013-02-01 14:19:00,NaT,NaT,Vacant,7,723196,2033 W NORTH AVE,EASTERWOOD,<NA>,I,"{""x"": 1411130.6501951367, ""y"": 598471.04346689..."
3,2106499A,0001 008,2022-05-13 16:20:00,NaT,NaT,Vacant,7,723197,2031 W NORTH AVE,EASTERWOOD,<NA>,I,"{""x"": 1411144.2272677273, ""y"": 598471.67732389..."
4,2078873A,0001 013,2022-02-10 16:01:00,NaT,NaT,Vacant,7,723202,2021 W NORTH AVE,EASTERWOOD,<NA>,I,"{""x"": 1411214.1001594663, ""y"": 598474.91288173..."


In [42]:
#CREATE A VACANCY SPECIFIC QUERY FILTER
#Set query specifications so only pull props with blocklots in vacancyDF
blocklots = vacancyDF["BLOCKLOT"] #blocklots are strings
blocklots = [f"'{blocklot}'" for blocklot in blocklots ] #add single quotes around strings so labeled as strings in query
blocklots = ', '.join(blocklots)
vacantOnly_query =f"BLOCKLOT IN ({blocklots})"

In [71]:
#PROPERTY LAYER

#Connect to feature layer of interest and view fields
url = 'https://geodata.baltimorecity.gov/egis/rest/services/Housing/dmxOwnership/MapServer/0'
property_feature_layer = FeatureLayer(url)
props = property_feature_layer.properties

maxRecs=20000

#propertyDF = pd.DataFrame()
queryOffset = 0
maxQuerySize = 1000
propertyDF=pd.DataFrame()

while propertyDF.shape[0] < maxRecs:
  query_res = property_feature_layer.query(where=vacantOnly_query, out_fields="*", result_record_count = maxQuerySize, return_all_records = False, result_offset=queryOffset)
  print(f'{query_res.df.shape[0]=}')
  propertyDF = pd.concat([propertyDF, query_res.df], ignore_index=True)
  print(f'{propertyDF.shape[0]}')
  if query_res.df.shape[0] < maxQuerySize:
    break
  queryOffset += maxQuerySize

if propertyDF.shape[0] >= maxRecs:
  print('WARNING: MAY BE MISSING SOME ENTRIES BECAUSE QUERY LIMIT REACHED (increase maxRecs to avoid this.')
  print('Note: BLOCKLOT uniquely identifies a lot by its block number and lot number on that block')

print('unique blocklots: ' + str(len(propertyDF['BLOCKLOT'].unique())))
print('total entries: ' + str(len(propertyDF['BLOCKLOT'])))

query_res.df.shape[0]=1000
1000
query_res.df.shape[0]=1000
2000
query_res.df.shape[0]=1000
3000
query_res.df.shape[0]=1000
4000
query_res.df.shape[0]=1000
5000
query_res.df.shape[0]=1000
6000
query_res.df.shape[0]=1000
7000
query_res.df.shape[0]=1000
8000
query_res.df.shape[0]=1000
9000
query_res.df.shape[0]=1000
10000
query_res.df.shape[0]=1000
11000
query_res.df.shape[0]=1000
12000
query_res.df.shape[0]=1000
13000
query_res.df.shape[0]=728
13728
unique blocklots: 13719
total entries: 13728


In [75]:
#PROPERTY TAX LIEN LAYER

#Connect to feature layer of interest and view fields
url = 'https://geodata.baltimorecity.gov/egis/rest/services/Housing/dmxLandTransactions/MapServer/10'
tax_lien_feature_layer = FeatureLayer(url)
layerProps = tax_lien_feature_layer.properties
print(f'{layerProps.name} (CodeMap Layer: {layerProps.id})')

#Create dataframe to store data of interest
colsOfInterest = ['LIEN_DATA_SOURCE',
                  'BLOCKLOT',
                  'TAX_SALE_YEAR',
                  'SALE_DATE',
                  'LIEN_DESCRIPTION1',
                  'LIEN_DESC2',
                  'REDEM_DATE',
                  'TOTAL_LIENS',
                  'BIDDER' ]

tax_lienDF = pd.DataFrame(columns = colsOfInterest)

#Get Data from all sublayers
tax_lien_sublayers = layerProps.subLayers
for sublayer in tax_lien_sublayers:
  #Get sublayer
  url = 'https://geodata.baltimorecity.gov/egis/rest/services/Housing/dmxLandTransactions/MapServer/' + str(sublayer["id"])
  tax_lien_feature_sublayer = FeatureLayer(url)
  layerProps = tax_lien_feature_sublayer.properties
  print(f'{layerProps.name} (CodeMap Layer: {layerProps.id})')

  # Query the layer to get all features (records)
  maxRecs = 100000
  queryOffset = 0
  sublayerDF = pd.DataFrame()
  while propertyDF.shape[0] < maxRecs:
    maxQuerySize = 1000
    query_res = tax_lien_feature_sublayer.query(where=vacantOnly_query, out_fields="*", result_record_count = maxQuerySize, return_all_records = False, result_offset=queryOffset)
    print(f'{query_res.df.shape[0]=}')
    sublayerDF = pd.concat([sublayerDF, query_res.df], ignore_index=True)
    print(f'{sublayerDF.shape[0]}')
    if query_res.df.shape[0] < maxQuerySize:
      break
    queryOffset += maxQuerySize

  #Print sublayer result info
  sublayerCount = sublayerDF.shape[0]
  print(sublayerCount)
  print(sublayerDF.info())
  print("=============================")

  #Add sublayer data to layer dataframe
  sublayerDF['LIEN_DATA_SOURCE'] = layerProps.name #Add layer name to dataframe
  tax_lienDF = pd.concat((tax_lienDF, sublayerDF[colsOfInterest]), ignore_index=True)

print('unique blocklots: ' + str(len(tax_lienDF['BLOCKLOT'].unique())))
print('total entries: ' + str(len(tax_lienDF['BLOCKLOT'])))
tax_lienDF.head()



Tax Certificates, Active (CodeMap Layer: 10)
Bulk 2022, MCC (City Interest) (CodeMap Layer: 11)
query_res.df.shape[0]=1000
1000
query_res.df.shape[0]=32
1032
1032
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   OBJECTID           1032 non-null   Int64   
 1   BLOCK_NO           1032 non-null   string  
 2   LOT_NO             1032 non-null   string  
 3   BLOCKLOT           1032 non-null   string  
 4   TAX_SALE_YEAR      1032 non-null   Int32   
 5   SALE_DATE          1032 non-null   string  
 6   LIEN_DESCRIPTION1  1032 non-null   string  
 7   LIEN_DESC2         1032 non-null   string  
 8   REDEM_DATE         1032 non-null   string  
 9   TOTAL_LIENS        1032 non-null   Float64 
 10  BIDDER             1032 non-null   string  
 11  SHAPE              1032 non-null   geometry
dtypes: Float64(1), Int32(1), Int64(1), geom

<ipython-input-75-8afe86ef295e>:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tax_lienDF = pd.concat((tax_lienDF, sublayerDF[colsOfInterest]), ignore_index=True)


query_res.df.shape[0]=914
914
914
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   OBJECTID           914 non-null    Int64   
 1   BLOCK_NO           914 non-null    string  
 2   LOT_NO             914 non-null    string  
 3   BLOCKLOT           914 non-null    string  
 4   TAX_SALE_YEAR      914 non-null    Int32   
 5   SALE_DATE          914 non-null    string  
 6   LIEN_DESCRIPTION1  914 non-null    string  
 7   LIEN_DESC2         914 non-null    string  
 8   REDEM_DATE         914 non-null    string  
 9   TOTAL_LIENS        914 non-null    Float64 
 10  BIDDER             914 non-null    string  
 11  SHAPE              914 non-null    geometry
dtypes: Float64(1), Int32(1), Int64(1), geometry(1), string(8)
memory usage: 84.9 KB
None
Bulk 2021, MCC (City Interest) (CodeMap Layer: 13)
query_res.df.shape[0]=353
353
353


,LIEN_DATA_SOURCE,BLOCKLOT,TAX_SALE_YEAR,SALE_DATE,LIEN_DESCRIPTION1,LIEN_DESC2,REDEM_DATE,TOTAL_LIENS,BIDDER
0,"Bulk 2022, MCC (City Interest)",1829 045,22,101922,R/P 2022/23 2021/22 ...,ECS ...,0,689737.0,MAYOR AND CITY COUNCIL OF BALTIMORE
1,"Bulk 2022, MCC (City Interest)",1540 062,22,101922,T/S 102319 5-M/B ...,...,0,1892172.0,MAYOR AND CITY COUNCIL OF BALTIMORE
2,"Bulk 2022, MCC (City Interest)",3408 008,22,101922,R/P 2022/23 2021/22 2020/21 T/S 102319 6-M/B ...,...,0,19821979.0,MAYOR AND CITY COUNCIL OF BALTIMORE
3,"Bulk 2022, MCC (City Interest)",1555 038,22,101922,R/P 2022/23 2021/22 2020/21 T/S 051319 5-M/B ...,REG ECS ...,0,6243194.0,MAYOR AND CITY COUNCIL OF BALTIMORE
4,"Bulk 2022, MCC (City Interest)",0279 092,22,101922,R/P 2022/23 2021/22 2020/21 T/S 102319 ...,ECS ...,0,7164014.0,MAYOR AND CITY COUNCIL OF BALTIMORE


In [77]:
#DEAL WITH LIEN DF DUPLICATES
import numpy as np

print('unique blocklots: ' + str(len(tax_lienDF['BLOCKLOT'].unique())))
print('total entries: ' + str(len(tax_lienDF['BLOCKLOT'])))

# Define custom aggregation dictionary
aggregations = {
    'LIEN_DATA_SOURCE': lambda x: ' || '.join(x), #concatenate with a || divider between entries
    'TAX_SALE_YEAR': lambda x: ' || '.join(map(str, x)),
    'SALE_DATE': lambda x: ' || '.join(x),
    'LIEN_DESCRIPTION1': lambda x: ' || '.join(x),
    'LIEN_DESC2': lambda x: ' || '.join(x),
    'REDEM_DATE': lambda x: ' || '.join(x),
    'TOTAL_LIENS': lambda x: ' || '.join(map(str, x)),
    'BIDDER': lambda x: ' || '.join(x)
}

# Group by 'blocklot' and apply the aggregation function above for all other columns
tax_lienDF = tax_lienDF.groupby('BLOCKLOT', as_index=False).agg(aggregations)

#Summarize Results
print(f'{tax_lienDF.shape}')
tax_lienDF.head()


unique blocklots: 4457
total entries: 4457
(4457, 9)


,BLOCKLOT,LIEN_DATA_SOURCE,TAX_SALE_YEAR,SALE_DATE,LIEN_DESCRIPTION1,LIEN_DESC2,REDEM_DATE,TOTAL_LIENS,BIDDER
0,0001 003,"Regular 2021, MCC (No Private Bidder) || All A...",21 || 21,51721 || 51721,R/P 2020/21 2019/20 2018/19 FWY T/S 051418 2-...,REG ECS ...,0 || 0,4976725.0 || 49767.25,MAYOR AND CITY COUNCIL OF BALTIMORE || MAYOR A...
1,0001 026,"Regular 2022, MCC (No Private Bidder)",22,62722,R/P 2021/22 2020/21 2019/20 T/S 051319 7-M/B ...,REG ...,0,6287978.0,MAYOR AND CITY COUNCIL OF BALTIMORE
2,0001 028,"Regular 2021, MCC (No Private Bidder) || All A...",21 || 21,51721 || 51721,R/P 2020/21 2019/20 2018/19 T/S 051418 2-R/P ...,REG ...,0 || 0,3547915.0 || 35479.15,MAYOR AND CITY COUNCIL OF BALTIMORE || MAYOR A...
3,0001 033,"Regular 2022, MCC (No Private Bidder)",22,62722,R/P 2021/22 2020/21 2019/20 T/S 051319 ...,REG ECS ...,0,6791857.0,MAYOR AND CITY COUNCIL OF BALTIMORE
4,0001 033C,"Regular 2021, MCC (No Private Bidder) || All A...",21 || 21,51721 || 51721,R/P 2020/21 2019/20 2018/19 T/S 051418 2-R/P ...,...,0 || 0,909747.0 || 9097.47,MAYOR AND CITY COUNCIL OF BALTIMORE || MAYOR A...


In [79]:
#JOIN TO CREATE FULL DF WITH COLUMNS OF INTEREST

vacancyCols = ["BLOCKLOT", "NT", "Address"] #NT is vacancy status
propertyCols = ["BLOCKLOT", "FULLADDR", 'SALEPRIC', 'SALEDATE', 'OWNER_ABBR', 'OWNER_1', 'OWNER_2', 'OWNER_3', 'STATETAX', 'CITY_TAX']
lienCols = ["BLOCKLOT", "TOTAL_LIENS", "LIEN_DATA_SOURCE"]

fullData = pd.merge(vacancyDF[vacancyCols], propertyDF[propertyCols], on='BLOCKLOT', how='left')
fullData = pd.merge(fullData, tax_lienDF[lienCols], on='BLOCKLOT', how='left')
fullData.shape
fullData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13759 entries, 0 to 13758
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BLOCKLOT          13759 non-null  string 
 1   NT                13759 non-null  string 
 2   Address           13759 non-null  string 
 3   FULLADDR          13728 non-null  string 
 4   SALEPRIC          13728 non-null  Int32  
 5   SALEDATE          13728 non-null  string 
 6   OWNER_ABBR        1121 non-null   string 
 7   OWNER_1           13728 non-null  string 
 8   OWNER_2           13728 non-null  string 
 9   OWNER_3           13728 non-null  string 
 10  STATETAX          13728 non-null  Float64
 11  CITY_TAX          13728 non-null  Float64
 12  TOTAL_LIENS       4458 non-null   object 
 13  LIEN_DATA_SOURCE  4458 non-null   object 
dtypes: Float64(2), Int32(1), object(2), string(9)
memory usage: 1.5+ MB


In [80]:
fullData["LIEN_DATA_SOURCE"].value_counts()

LIEN_DATA_SOURCE
Regular 2021, MCC (No Private Bidder) || All Active Tax Certificates                                                              1339
Bulk 2022, MCC (City Interest)                                                                                                    1026
Regular 2022, MCC (No Private Bidder)                                                                                              914
All Active Tax Certificates                                                                                                        477
Bulk 2021, MCC (City Interest) || All Active Tax Certificates                                                                      351
Private Bidder and Unredeemed || All Active Tax Certificates                                                                       341
Bulk 2022, MCC (City Interest) || Private Bidder and Unredeemed || All Active Tax Certificates                                       5
Private Bidder and Unredeemed || Priva

In [81]:
#PREVIEW DATA
fullData.head()

,BLOCKLOT,NT,Address,FULLADDR,SALEPRIC,SALEDATE,OWNER_ABBR,OWNER_1,OWNER_2,OWNER_3,STATETAX,CITY_TAX,TOTAL_LIENS,LIEN_DATA_SOURCE
0,0001 003,Vacant,2041 W NORTH AVE,2041 W NORTH AVE,0,10171975,<NA>,NEW YORK INC,,,10.08,202.32,4976725.0 || 49767.25,"Regular 2021, MCC (No Private Bidder) || All A..."
1,0001 004,Vacant,2039 W NORTH AVE,2039 W NORTH AVE,0,11132014,<NA>,"MUHAMMAD, ANNA AVON",,,10.08,202.32,NaN,NaN
2,0001 007,Vacant,2033 W NORTH AVE,2033 W NORTH AVE,2120,06072006,<NA>,"NORTH AVENUE REVELATION PROJECT,",LLC.,,27.25,547.01,NaN,NaN
3,0001 008,Vacant,2031 W NORTH AVE,2031 W NORTH AVE,2000,12152021,<NA>,VERONA ENTERPRISE LLC,,,10.08,202.32,NaN,NaN
4,0001 013,Vacant,2021 W NORTH AVE,2021 W NORTH AVE,8000,03312020,<NA>,"BRADLEY, GREGORY",,,10.08,202.32,NaN,NaN


In [ ]:
#CLEAN DATA

fullData[['OWNER_1', 'OWNER_2', 'OWNER_3']] = fullData[['OWNER_1', 'OWNER_2', 'OWNER_3']].replace(r'^\s*$', '_', regex=True) #Replace the entires that look blank but are actually a bunch of spaces with a single _
fullData.head()
print(f'{fullData["OWNER_2"][0] = }')

In [ ]:
columnNames = ["Vacant", "Condemned", "Address", "OwnerNames1", "LastSalePrice", "LastSaleDate", "LienAtSale" ]
codemapDf = pd.DataFrame(columns = columnNames)
codemapDf["Vacant"] = fullData["NT"]
codemapDf["Address"] = fullData["Address"]
codemapDf["OwnerNames1"] = fullData['OWNER_1'] + fullData['OWNER_2'] + fullData['OWNER_3']
codemapDf["LastSalePrice"]=fullData['SALEPRIC']
codemapDf["LastSaleDate"]=fullData['SALEDATE']


codemapDf.head()